<a href="https://colab.research.google.com/github/Braelin2/ECGR5105HW5/blob/main/ECGR5105_HW5_P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch as torch
import tensorflow as tf
import torch.optim as optim


from google.colab import drive
drive.mount('/content/drive')

In [7]:
file_path = '/content/drive/My Drive/ECGR5105/Housing.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))

In [8]:
#Applying a binary map
convlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Defining the map function
def Convert(x):
    return x.map({'yes': 1, 'no': 0, 'furnished':  1, 'semi-furnished':  0, 'unfurnished':  -1})

dataset[convlist] = dataset[convlist].apply(Convert)

In [9]:
def model(t_u, w5, w4, w3, w2, w1, b):
    return (w5*t_u**5) + (w4*t_u**4) + (w3*t_u**3) + (w2*t_u**2) + (w1*t_u) + b

In [10]:
def loss_fn(t_p, t_c):
  squared_diffs = (t_p - t_c)**2
  return squared_diffs.mean()

In [11]:
def training_loop(n_epochs, optimizer, params, t_u_train, t_u_test, t_c_train, t_c_test):
  for epoch in range(1, n_epochs + 1):
    if params.grad is not None:
      params.grad.zero_()

    t_p_train = model(t_u_train, *params)
    train_loss = loss_fn(t_p_train.transpose(0,1) , t_c_train)

    t_p_test = model(t_u_test, *params)
    test_loss = loss_fn(t_p_test.transpose(0,1), t_c_test)

    optimizer.zero_grad()
    train_loss.backward()
    test_loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
     print('Epoch %d, Training Loss %d, Testing Loss %f' % (epoch, float(train_loss), float(test_loss)))
  return params

In [13]:
from sklearn.model_selection import train_test_split

np.random.seed(0)
training, testing = train_test_split(dataset, train_size = 0.8, test_size = 0.2, random_state = 100)

Y_train = training.pop("price")
Y_test = testing.pop("price")

varlist =  ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus']

In [14]:
from sklearn.preprocessing import StandardScaler
standScaler = StandardScaler()

X_train_temp = training
X_train_temp[varlist] = standScaler.fit_transform(training[varlist])

X_test_temp = testing
X_test_temp[varlist] = standScaler.fit_transform(testing[varlist])

num_train = len(X_train_temp)
num_test = len(X_test_temp)

X_train_temp1 = np.c_[np.ones((num_train, 1)), X_train_temp[varlist]]
X_test_temp1 = np.c_[np.ones((num_test, 1)), X_test_temp[varlist]]

X_train_tensor = torch.tensor(X_train_temp1)
X_test_tensor = torch.tensor(X_test_temp1)

Y_train_tensor = torch.tensor(Y_train.values)
Y_test_tensor = torch.tensor(Y_test.values)

Y_train_normal = (Y_train_tensor - Y_train_tensor.float().mean())/Y_train.std()
Y_test_normal = (Y_test_tensor - Y_test_tensor.float().mean())/Y_test.std()

In [24]:
# SGD Optimizer:
# Learning Rate 0.00000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-8
optimizer = optim.SGD([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 26459096035097, Testing Loss 25143264865888.750000
Epoch 1000, Training Loss 26444845052613, Testing Loss 25132358368028.679688
Epoch 1500, Training Loss 26430710470895, Testing Loss 25121518152440.433594
Epoch 2000, Training Loss 26416691335785, Testing Loss 25110743763004.058594
Epoch 2500, Training Loss 26402786597260, Testing Loss 25100034669347.574219
Epoch 3000, Training Loss 26388995417359, Testing Loss 25089390491732.222656
Epoch 3500, Training Loss 26375316710634, Testing Loss 25078810637348.027344
Epoch 4000, Training Loss 26361749520358, Testing Loss 25068294669647.847656
Epoch 4500, Training Loss 26348293052148, Testing Loss 25057842211578.296875
Epoch 5000, Training Loss 26334946275589, Testing Loss 25047452741661.707031


tensor([-274.0860, 3027.1482, 1072.3868,  979.4740,  191.0082,  943.2203],
       requires_grad=True)

In [25]:
# SGD Optimizer:
# Learning Rate 0.00000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-9
optimizer = optim.SGD([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 26471996416988, Testing Loss 25153117893571.453125
Epoch 1000, Training Loss 26470555370290, Testing Loss 25152018340510.398438
Epoch 1500, Training Loss 26469115611994, Testing Loss 25150919342884.179688
Epoch 2000, Training Loss 26467677036136, Testing Loss 25149821031827.574219
Epoch 2500, Training Loss 26466239628482, Testing Loss 25148723369816.101562
Epoch 3000, Training Loss 26464803404318, Testing Loss 25147626397670.937500
Epoch 3500, Training Loss 26463368383026, Testing Loss 25146530105929.167969
Epoch 4000, Training Loss 26461934517660, Testing Loss 25145434464764.566406
Epoch 4500, Training Loss 26460501891860, Testing Loss 25144339553108.066406
Epoch 5000, Training Loss 26459070391654, Testing Loss 25143245257171.273438


tensor([ -2.3934, 311.7432, 110.0760,  99.5430,  20.1588,  94.4241],
       requires_grad=True)

In [26]:
# SGD Optimizer:
# Learning Rate 0.0000000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-10
optimizer = optim.SGD([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 26474674450914, Testing Loss 25153703364291.375000
Epoch 1000, Training Loss 26473631102372, Testing Loss 25153632547562.726562
Epoch 1500, Training Loss 26473172393810, Testing Loss 25153731581153.527344
Epoch 2000, Training Loss 26472918111244, Testing Loss 25153718674942.488281
Epoch 2500, Training Loss 26472735376785, Testing Loss 25153645571912.632812
Epoch 3000, Training Loss 26472577689264, Testing Loss 25153548830678.539062
Epoch 3500, Training Loss 26472428775992, Testing Loss 25153443503094.031250
Epoch 4000, Training Loss 26472282945351, Testing Loss 25153335136538.996094
Epoch 4500, Training Loss 26472138188453, Testing Loss 25153225705167.769531
Epoch 5000, Training Loss 26471993858502, Testing Loss 25153115902242.468750


tensor([25.3401, 34.5134, 12.1553, 10.8830,  2.9196,  9.4436],
       requires_grad=True)

In [27]:
# SGD Optimizer:
# Learning Rate 0.00000000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-11
optimizer = optim.SGD([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 26476976950461, Testing Loss 25157107992377.652344
Epoch 1000, Training Loss 26476607736151, Testing Loss 25156229878219.242188
Epoch 1500, Training Loss 26476273991027, Testing Loss 25155547459539.578125
Epoch 2000, Training Loss 26475972157173, Testing Loss 25155020565141.949219
Epoch 2500, Training Loss 26475699034191, Testing Loss 25154616933177.101562
Epoch 3000, Training Loss 26475451745945, Testing Loss 25154310689842.507812
Epoch 3500, Training Loss 26475227704546, Testing Loss 25154081108810.621094
Epoch 4000, Training Loss 26475024584609, Testing Loss 25153911616059.699219
Epoch 4500, Training Loss 26474840291624, Testing Loss 25153788977010.265625
Epoch 5000, Training Loss 26474672945494, Testing Loss 25153702649276.753906


tensor([18.5082,  5.8057,  2.2565,  2.0022,  1.1934,  0.9445],
       requires_grad=True)

In [28]:
# ADAM Optimizer
# Learning Rate 0.1
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-1
optimizer = optim.Adam([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 26470378502478, Testing Loss 25151671369199.375000
Epoch 1000, Training Loss 26467117940590, Testing Loss 25149174476759.621094
Epoch 1500, Training Loss 26463801491843, Testing Loss 25146443157525.039062
Epoch 2000, Training Loss 26460402078713, Testing Loss 25143665149834.132812
Epoch 2500, Training Loss 26456961927401, Testing Loss 25140865167504.164062
Epoch 3000, Training Loss 26453501183149, Testing Loss 25138054996586.773438
Epoch 3500, Training Loss 26450030129848, Testing Loss 25135240441886.683594
Epoch 4000, Training Loss 26446554535130, Testing Loss 25132424670188.167969
Epoch 4500, Training Loss 26443077672858, Testing Loss 25129609345654.093750
Epoch 5000, Training Loss 26439601182137, Testing Loss 25126795151518.476562


tensor([-22.5051, 476.6408, 493.5866, 500.0357, 500.2957, 499.9173],
       requires_grad=True)

In [29]:
# ADAM Optimizer
# Learning Rate 0.001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-3
optimizer = optim.Adam([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 26477273012280, Testing Loss 25157968053471.960938
Epoch 1000, Training Loss 26477161782936, Testing Loss 25157714505614.828125
Epoch 1500, Training Loss 26477051152597, Testing Loss 25157468679913.859375
Epoch 2000, Training Loss 26476941012699, Testing Loss 25157230179810.855469
Epoch 2500, Training Loss 26476831271950, Testing Loss 25156998679274.203125
Epoch 3000, Training Loss 26476721852292, Testing Loss 25156773912925.414062
Epoch 3500, Training Loss 26476612693847, Testing Loss 25156555675615.007812
Epoch 4000, Training Loss 26476503748007, Testing Loss 25156343806826.273438
Epoch 4500, Training Loss 26476394979663, Testing Loss 25156138189234.347656
Epoch 5000, Training Loss 26476286364569, Testing Loss 25155938733836.046875


tensor([5.8534, 5.9353, 5.9737, 5.9964, 5.9981, 4.9998], requires_grad=True)

In [30]:
# ADAM Optimizer
# Learning Rate 0.0001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-4
optimizer = optim.Adam([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 26477373528716, Testing Loss 25158202755189.843750
Epoch 1000, Training Loss 26477362306788, Testing Loss 25158176287667.140625
Epoch 1500, Training Loss 26477351093105, Testing Loss 25158149907952.066406
Epoch 2000, Training Loss 26477339880226, Testing Loss 25158123592166.515625
Epoch 2500, Training Loss 26477328674542, Testing Loss 25158097361147.878906
Epoch 3000, Training Loss 26477317470273, Testing Loss 25158071197055.023438
Epoch 3500, Training Loss 26477306268003, Testing Loss 25158045098612.960938
Epoch 4000, Training Loss 26477295066916, Testing Loss 25158019064710.785156
Epoch 4500, Training Loss 26477283865942, Testing Loss 25157993093903.175781
Epoch 5000, Training Loss 26477272665082, Testing Loss 25157967186190.121094


tensor([1.4987, 1.4993, 1.4997, 1.5001, 1.5001, 0.5000], requires_grad=True)

In [31]:
# ADAM Optimizer
# Learning Rate 0.00001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-5
optimizer = optim.Adam([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 26477383613144, Testing Loss 25158226593564.648438
Epoch 1000, Training Loss 26477382488499, Testing Loss 25158223932082.464844
Epoch 1500, Training Loss 26477381363856, Testing Loss 25158221271238.328125
Epoch 2000, Training Loss 26477380239210, Testing Loss 25158218611027.253906
Epoch 2500, Training Loss 26477379114565, Testing Loss 25158215951452.464844
Epoch 3000, Training Loss 26477377989921, Testing Loss 25158213292514.507812
Epoch 3500, Training Loss 26477376865284, Testing Loss 25158210634219.882812
Epoch 4000, Training Loss 26477375740652, Testing Loss 25158207976564.257812
Epoch 4500, Training Loss 26477374616020, Testing Loss 25158205319545.468750
Epoch 5000, Training Loss 26477373491389, Testing Loss 25158202663163.515625


tensor([1.0501, 1.0501, 1.0501, 1.0501, 1.0501, 0.0500], requires_grad=True)